In [1]:
library(tidyverse)
library(Seurat)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Attaching SeuratObject

Attaching sp



In [2]:
other_clusters = qs::qread('../00_preprocessing/_targets/objects/exp_labelled_other') %>% `[[` %>% pull(labels) %>% unique
neuron_clusters = qs::qread('../00_preprocessing/_targets/objects/exp_RElabelled_neuron') %>% `[[` %>% pull(labels) %>% unique

In [3]:
day_splits = tibble(object = rlang::syms(c("exp_labelled_other", "exp_labelled_neuron")))
days = tibble(day = c("Day5", "Day14", "all"))
day_splits = tidyr::crossing(day_splits, days)
# day_splits = day_splits %>% rowwise %>% mutate(day_label)
day_splits

object,day
<list>,<chr>
exp_labelled_other,all
exp_labelled_other,Day14
exp_labelled_other,Day5
exp_labelled_neuron,all
exp_labelled_neuron,Day14
exp_labelled_neuron,Day5


In [4]:
other_clusters_tibble = tibble(label = other_clusters) %>% 
tidyr::crossing(day_splits) %>%
filter(str_detect(as.character(object), 'other'))


neuron_clusters_tibble = tibble(label = neuron_clusters) %>% 
tidyr::crossing(day_splits) %>%
filter(str_detect(as.character(object), 'neuron'))

clusters_tibble = rbind(other_clusters_tibble, neuron_clusters_tibble)
clusters_tibble = clusters_tibble %>% 
mutate(cluster = label) %>%
rowwise %>%
mutate(label = case_when(day != "all" ~ paste0(c(day, as.character(label)), collapse='.'),
                          TRUE ~ paste0(label, collapse='.')))
clusters_tibble

label,object,day,cluster
<chr>,<list>,<chr>,<fct>
Astrocytes,exp_labelled_other,all,Astrocytes
Day14.Astrocytes,exp_labelled_other,Day14,Astrocytes
Day5.Astrocytes,exp_labelled_other,Day5,Astrocytes
a1_Tany-Astrocytes,exp_labelled_other,all,a1_Tany-Astrocytes
Day14.a1_Tany-Astrocytes,exp_labelled_other,Day14,a1_Tany-Astrocytes
Day5.a1_Tany-Astrocytes,exp_labelled_other,Day5,a1_Tany-Astrocytes
MOL,exp_labelled_other,all,MOL
Day14.MOL,exp_labelled_other,Day14,MOL
Day5.MOL,exp_labelled_other,Day5,MOL


In [5]:
clusters_tibble$cluster = clusters_tibble %>% pull(cluster) %>% stringr::str_replace(fixed('-'), '__')
clusters_tibble$label = clusters_tibble %>% pull(label) %>% stringr::str_replace(fixed('-'), '__')


In [6]:
clusters_tibble

label,object,day,cluster
<chr>,<list>,<chr>,<chr>
Astrocytes,exp_labelled_other,all,Astrocytes
Day14.Astrocytes,exp_labelled_other,Day14,Astrocytes
Day5.Astrocytes,exp_labelled_other,Day5,Astrocytes
a1_Tany__Astrocytes,exp_labelled_other,all,a1_Tany__Astrocytes
Day14.a1_Tany__Astrocytes,exp_labelled_other,Day14,a1_Tany__Astrocytes
Day5.a1_Tany__Astrocytes,exp_labelled_other,Day5,a1_Tany__Astrocytes
MOL,exp_labelled_other,all,MOL
Day14.MOL,exp_labelled_other,Day14,MOL
Day5.MOL,exp_labelled_other,Day5,MOL


In [7]:
errored_labels = targets::tar_meta() %>%
    filter(!is.na(error)) %>% 
    select(name) %>%
    rowwise %>%
    filter(str_detect(name, 'da_results_00_')) %>%
    mutate(name = str_split(name, '___', n=2)[[1]][1]) %>%
    mutate(name = str_replace(name, 'da_results_00_', '')) %>%
    distinct %>% 
    pull(name)
errored_labels

[1] "Day14.Endothelial"                "Day14.Fibroblasts2__Fibroblasts3"
 [3] "Day14.MOL__a2_Tanycytes"          "Day14.Parstuber2A__ParsTuber1"   
 [5] "Day14.Unassigned2__Rgs16_Vip"     "Day5.Endothelial"                
 [7] "Day5.MOL__a2_Tanycytes"           "Day5.Parstuber2A__ParsTuber1"    
 [9] "Day5.Unassigned2__Unassigned1"    "Endothelial"                     
[11] "MOL__a2_Tanycytes"                "Parstuber2A__ParsTuber1"         
[13] "Day14.Mural_Cells2"               "Day5.Unassigned2__Rgs16_Vip"     
[15] "Day5.Mural_Cells2"

In [8]:
clusters_tibble = clusters_tibble %>%
filter(!(label %in% errored_labels))

clusters_tibble

label,object,day,cluster
<chr>,<list>,<chr>,<chr>
Astrocytes,exp_labelled_other,all,Astrocytes
Day14.Astrocytes,exp_labelled_other,Day14,Astrocytes
Day5.Astrocytes,exp_labelled_other,Day5,Astrocytes
a1_Tany__Astrocytes,exp_labelled_other,all,a1_Tany__Astrocytes
Day14.a1_Tany__Astrocytes,exp_labelled_other,Day14,a1_Tany__Astrocytes
Day5.a1_Tany__Astrocytes,exp_labelled_other,Day5,a1_Tany__Astrocytes
MOL,exp_labelled_other,all,MOL
Day14.MOL,exp_labelled_other,Day14,MOL
Day5.MOL,exp_labelled_other,Day5,MOL


In [9]:
qs::qsave(clusters_tibble, 'clusters_tibble.qs')